In [13]:
# boto3 s3 operations
#List
#%pip install boto3

import os
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')

import boto3

def list_buckets():
    s3 = boto3.resource('s3')
    for bucket in s3.buckets.all():
        print(bucket.name)

# Call the function to list buckets
list_buckets()


abdobucketcicd
classgptbucket


In [4]:
from dotenv import load_dotenv

# load OPENAI API KEY
load_dotenv()
import os
res = os.getenv('AWS_ACCESS_KEY_ID')

In [5]:
res

In [18]:
def list_folders():
    s3 = boto3.resource('s3')
    bucket = s3.Bucket("classgptbucket")
    folders = set()
    for obj in bucket.objects.all():
        # split the key by / and take the first element
        folders.add(obj.key.split("/")[0])
        
    return folders
list_folders()

{'PDF1', 'pdf2'}

In [20]:
# List files

from collections import defaultdict
from pprint import pprint

def list_files():
    s3 = boto3.resource('s3')
    bucket = s3.Bucket("classgptbucket")
    
    classes = defaultdict(list)
    
    for obj in bucket.objects.filter():
        cname , fname = obj.key.split("/")
        if not fname.endswith(".json"):
            classes[cname].append(fname)
    return classes
pprint(list_files())

defaultdict(<class 'list'>,
            {'PDF1': ['', 'Internships_2023_1688485790.pdf', 'MY_RES.pdf'],
             'pdf2': ['',
                      'Abdelrahman Youssef Mohamed Ahmed.pdf',
                      'Credit_Card_Number_Fraud_Detection_Using.pdf',
                      'عبدالله يوسف.pdf']})


In [21]:
# Check if folder exist

def folder_exists(folder_name):
    s3 = boto3.resource("s3")
    bucket = s3.Bucket('classgptbucket')
    
    for _ in bucket.objects.filter(Prefix=f"{folder_name}/"):
        return True
    return False
folder_exists('pdf2')

True

In [29]:
def folder_exists_bucket(bucket,folder_name):
    s3 = boto3.resource("s3")
    #bucket = s3.Bucket(bucket)
    
    for _ in bucket.objects.filter(Prefix=f"{folder_name}/"):
        return True
    return False

In [22]:
folder_exists("tt")

False

In [23]:
# check files if exists

import botocore

def file_exists(folder_name , file_name):
    s3 = boto3.resource('s3')
    
    try:
        s3.Object("classgptbucket" , f"{folder_name}/{file_name}").load()
        return True
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            return False
        else:
            raise

folder_name = "pdf2"
file_name = "Abdelrahman Youssef Mohamed Ahmed.pdf"
file_exists(folder_name , file_name)

True

In [24]:
folder_name = "pdf"
file_name = "Abdelrahman Youssef Mohamed Ahmed.pdf"
file_exists(folder_name , file_name)

False

In [30]:
# Create folder

def create_folder(bucket,folder_name):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('classgptbucket')
    
    if not folder_exists_bucket(bucket,folder_name):
        bucket.put_object(Key=f"{folder_name}/")

create_folder('classgptbucket',"test")
list_folders()

{'PDF1', 'pdf2', 'test'}

In [31]:
# adding files
def upload_files(file_obj , file_path):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('classgptbucket')
    
    bucket.upload_fileobj(file_obj , file_path)

with open('/Users/pepo_abdo/Downloads/MY_RES.pdf' , "rb") as f:
    upload_files(f , 'test/abdo.pdf')

list_files()

defaultdict(list,
            {'PDF1': ['', 'Internships_2023_1688485790.pdf', 'MY_RES.pdf'],
             'pdf2': ['',
              'Abdelrahman Youssef Mohamed Ahmed.pdf',
              'Credit_Card_Number_Fraud_Detection_Using.pdf',
              'عبدالله يوسف.pdf'],
             'test': ['', 'abdo.pdf']})

In [32]:
# remove folder

def remove_folder(folder_name):
    
    s3 = boto3.resource("s3")
    bucket = s3.Bucket('classgptbucket')
    
    if folder_exists_bucket(bucket,folder_name):
        for key in bucket.objects.filter(Prefix=f"{folder_name}/"):
            key.delete()
remove_folder("test")
list_folders()

{'PDF1', 'pdf2'}

In [33]:
# create a dataclass titled s3 that implements the methods above

class S3:
    def __init__(self, bucket_name):
        self.bucket_name = bucket_name
        self.s3 = boto3.resource("s3")
        self.bucket = self.s3.Bucket(bucket_name)

    def list_folders(self):
        folders = set()
        for obj in self.bucket.objects.filter():
            folders.add(obj.key.split("/")[0])

        return folders

    def list_files(self):
        classes = defaultdict(list)

        # loop through only the parent directory
        for obj in self.bucket.objects.filter():
            cname, fname = obj.key.split("/")
            if not fname.endswith(".json"):
                classes[cname].append(fname)

        return classes

    def folder_exists(self, folder_name):
        for _ in self.bucket.objects.filter(Prefix=f"{folder_name}/"):
            return True
        return False

    def file_exists(self, folder_name, file_name):
        try:
            self.s3.Object(self.bucket_name, f"{folder_name}/{file_name}").load()
            return True
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                return False
            else:
                raise

    def create_folder(self, folder_name):
        if not self.folder_exists(folder_name):
            self.bucket.put_object(Key=f"{folder_name}/")

    def upload_files(self, file_obj, file_path):
        self.bucket.upload_fileobj(file_obj, file_path)

    def remove_folder(self, folder_name):
        if self.folder_exists(folder_name):
            for key in self.bucket.objects.filter(Prefix=f"{folder_name}/"):
                key.delete()

    def remove_file(self, folder_name, file_name):
        if self.folder_exists(folder_name):
            self.bucket.objects.filter(Prefix=f"{folder_name}/{file_name}").delete(
                Delete={"Objects": [{"Key": f"{folder_name}/{file_name}"}]}
            )